In [8]:
% Initial settings of the GNU Octave kernel€
warning('off'); graphics_toolkit('gnuplot'); format compact;

Tento markdown blok obsahuje definici Latexové makro `\Aop` pro MKP operátor sestavení globálních matic.
$\newcommand{\Aop}{\mathop{\vphantom{\sum}\vphantom{\rule{0pt}{2.5ex}}\smash{\lower{1.0ex}{\text{{\huge A}}}}}}$
$\newcommand{\Aop}{\displaystyle\mathop{\Large\mathrm{A}}\limits}$

## Disclaimer: jedná se o didaktický kód, ne produkční
- Zvolen jednoduchý programovací jazyk GNU Octave.
- Cílem je vysvětlit algoritmy na jednoduché implementaci.
- Bez snahy o solidní softwarový design.
- Bez optimalizace kódu.
- Chybí řádné ošetření chybových stavů a vyjímek.

## Řešení 2D úlohy lineární elasticity pomocí MKP
Vyjdeme ze slabé formulace (bez objemových sil):
$$
\int_{\Omega}\delta\boldsymbol{\varepsilon}:\mathcal{D}:\boldsymbol{\varepsilon}\,\text{d}V-\int_{\Gamma_{\text{N}}}\delta\boldsymbol{u}\cdot\bar{\boldsymbol{t}}\,\text{d}S=0
$$
a její MKP diskretizace:
$$
\delta\mathbf{u}^{\mathsf{T}}
\left\{ \left[\Aop_{el=1}^{n_{\text{el}}}\int_{\Omega_{el}}\mathbf{B}^{\mathsf{T}}\mathbf{D}\mathbf{B}\,\text{d}V_{el}\right]\mathbf{u}-\Aop_{el=1}^{n_{\text{el}}}\left[\int_{\Gamma_{\text{N}}^{e}}\mathbf{N}^{\mathsf{T}}\bar{\mathbf{t}}\,\text{d}S_{el}\right]\right\} =0
$$
která vede na lineární systém algebraický rovnic:
$$
\mathbf{K}\mathbf{u}-\mathbf{f}=\mathbf{0}
$$

Pro sestavení matice tuhosti a vektoru pravé strany:
$$
\mathbf{K}= 
\Aop_{el=1}^{n_\text{en}}
\sum_{g}\mathbf{B}_{g}^{\mathsf{T}}\mathbf{D}\mathbf{B}_{g}w_{g}\left\Vert J_{g}\right\Vert ,\qquad
\mathbf{f}=
\Aop_{el=1}^{n_\text{en}}
\sum_{g}\mathbf{N}^{\mathsf{T}}\bar{\mathbf{t}}_{g}w_{g}\left\Vert J_{g}^{\text{s}}\right\Vert 
$$
budeme potřebovat:
1. Tvorba sítě
2. Sestavení globání matice a vektoru
3. Gaussova inetgrace
4. Matice tvarových funkcí $\mathbf{N}$
5. Operátorová matice $\mathbf{B}$
6. Matice materiálové tuhosti $\mathbf{D}$
7. Okrajové podmínky
8. Řešení
9. Post-processing

## 1. Tvorba sítě

<table><tr>
<td style="width: 15%;">
    
![](assets/2D_bar_with_quads.svg)

</td>
<td style="width: 85%;">

Pro definovanou geometrii potřebujeme vytvořit výpočtovou síť. Síť v našém kódu budeme reprezentovat dvěma dvourozměrnými poli: polem souřadnic uzlů `X` a polem konektivity elementů `IEN`. Tyto pole můžeme pro jednoduché oblasti vytvořit ručně, ale pro složitější geometrie je užitečné použít tzv. síťovač, např. [Gmsh](https://gmsh.info).
    
<div class="llm-prompt">
<strong>LLM Prompt:</strong> Vytvoř mi vstupní soubor pro Gmsh `bar.geo`, pro 2D obdélník o délce L = 1 m a šířce b = sqrt(A) m, kde průřez A = 0.1 m^2. Počátek kartézských souřadnic zvol v levém horním rohu. Souřadnice b míří ve směru +x a L ve směru -y. Obdélník vysíťuj třemi lineárními quad elementy ve směru y.
</div>
</td>
</tr></table>

### Souřadnice uzlových bodů

In [9]:
nsd = 2; % Number of Spatial Dimensions
run("assets/bar.m"); % read msh data structure exported from gmsh
X = msh.POS(:,1:nsd)'




X =
        0   0.3162   0.3162        0   0.3162   0.3162        0        0
        0        0  -1.0000  -1.0000  -0.3333  -0.6667  -0.6667  -0.3333



### Pole konektivity


In [10]:
nen = 4; % Number of Element Nodes
IEN = msh.QUADS(:,1:nen)'



IEN =
   1   8   7
   2   5   6
   5   6   3
   8   7   4



## 2. Sestavení globání matice a vektoru

Na nejvyšší úrovni se provádí proces sestavení $\Aop_{el=1}^{n_\text{en}}$ globální matice tuhosti $\mathbf{K}$ a globálního vektoru pravé strany $\mathbf{f}$ z lokálních matic $\mathbf{K}_{el}$ a lokálních vektorů $\mathbf{f}_{el}$.

Nejprve si zvolíme, že neznámé složky posunutí pro jednotlivé uzly sítě budeme řadit ve vektoru řešní $\mathbf{u}$ takto:

$$
\begin{array}{c|ccccccccc}
i & 1 & 2 & 3 & 4 & 5 & 6 & \cdots & (n_{\text{np}}\times n_{\text{nd}})-1 & n_{\text{np}}\times n_{\text{nd}}\\
\hline
u_i & u_{1} & v_{1} & u_{2} & v_{2} & u_{3} & v_{3} & \cdots & u_{\text{nnp}} & v_{\text{nnp}}
\end{array}
$$

Vidíme, že délka vektoru $\mathbf{u}$ je součin počtu uzlů sítě `nnp` a počtu stupňů volnosti uzlu `nnd`, který je pro náš 2D případ roven 2.

Abychom se mohli snadněji odkazovat mezi globálními čísli uzlů $A\in\left[1,\ldots n_{\text{np}}\right]$ a čísly rovnic jim odpovídajícími $eq\in\left[1,\ldots n_{\text{eq}}\right]$, kde  $n_{\text{eq}}=n_{\text{np}}\times n_{\text{nd}}$, zavedeme další pomocné pole konektivity `ID` takto: 
$$
\mathtt{ID} = \left[\begin{array}{ccccc}
1 & 3 & 5 & \cdots & n_{\text{eq}} - 1 \\
2 & 4 & 6 & \ldots & n_{\text{eq}}
\end{array}\right]
$$
`A`-tý sloupec pole `ID` odpovídá `A`-tému globálnímu uzlu. Řádky v tomto `A`-tém sloupci obsahují odpovídající čísla neznámých složek vektoru posunutí $\mathbf{u}$.

Pole `ID` bychom mohli implementovat pomocí cyklu a indexové aritmetiky. Pro jednoduchost však využijeme vestavěné funkci `reshape()`, která 1D pole čísel rovnic převede na 2D pole, přesně jak jsme si definovali:

In [11]:
nnp = size(X,2);   % Number of Nodal Points
nnd = nsd;         % Number of Nodal DOFs =nsd=2(ux, uy)
neq = nnp * nnd;   % Number od EQuations
ID = reshape(1:nnp*nnd, nnd, [])





ID =
    1    3    5    7    9   11   13   15
    2    4    6    8   10   12   14   16



Následující fragment kódu ukazuje proces sestavení jako cyklus přes všechny elementy `nel`, ve kterém se z pole konektivity `IEN` získají globální čísla úzlů `A` elementu `el`. Pro ty se následně z pole `ID` získají čísla rovnic `eq`. Protože `ID` je dvourozměrné pole, `eq` je také dvourozměrné. Abychom jej mohli použít jako řez globální maticí a vektorem, použijeme opět funkci `reshape()` a převedeme jej na jednorozměrný vektor indexů. V závěru for cyklu už jen snadno **kumulujeme** příspěvky z lokální matice a vektoru.

In [12]:
K = sparse(neq, neq);

nel = size(IEN,2); % Number of ELement
for el = 1:nel
    Ke = zeros(nen*nnd,nen*nnd);
    
    A = IEN(:,el);
    
    eq = ID(:,A);
    eq = reshape(eq, [], 1);
    K(eq, eq) = K(eq, eq) + Ke;
end

## 3. Gaussova inetgrace

Pro výpočet lokální matice tuhosti $\mathbf{K}_{el}$ potřebujeme numerickou integraci přes objem elementu. Numerická integrace spočívá v aproximaci integrálu funkce sumací jejich funkční hodnot ve vhodně vybraných bodech vynásobených váhovými koeficienty těchto bodů

$$
\int_{\square}f(\boldsymbol{\xi})\,\text{d}\boldsymbol{\xi}\approx\sum_{g=1}^{n_{\text{gp}}}f(\boldsymbol{\xi}_{g})w_{g}
$$

kde $\square\equiv\left[-1,1\right]\times\left[-1,1\right]$. Přechod od integrační oblasti $\square$ k oblasti elementu $\Omega_{el}$ souvisí s transformací isoparametrických souřadnic $\boldsymbol{\xi}$ na fyzikální souřadnice $\boldsymbol{x}$ a pozornost mu bude věnována později.

Konkrétní polohy integračních bodů a jejich váhy závisí na typu a řádu integrace. V MKP se nejčastěji používá Gaussova-Legendreova integrace. Je přesná pro polynomy až do stupně $2n-1$, kde $n$ je počet uzlů 1D integrace. Pro definici opět využijeme LLM prompt.

**Prompt:**
```
Dej mi dvě řádkové matice `gp` a `gw`, zapsané v syntaxi GNU Octave, jejichž sloupce budou odpovídat souřadnicím a vahám 2D Gaussovy- Legendreovy integrace řádu optimálního pro čtyřuzlové izoparametrické bilineární quad elementy.
```

In [13]:
gp_bulk = [-1 1 1 -1;
           -1 -1 1 1] / sqrt(3);
gw_bulk = [1 1 1 1];
ngp = length(gw_bulk);

Zbývá zanořit cyklus přes integrační body do cyklu přes elementy. V těchto bodech potřebujeme vyčíslit tvarové funkce a jejich derivace. Jak na to si ukážeme dále.

In [14]:
K = sparse(neq, neq);

nel = size(IEN,2); % Number of ELement
for el = 1:nel
    Ke = zeros(nen*nnd,nen*nnd);

    A = IEN(:,el);
    
    for g = 1:ngp
       Xi_g = gp_bulk(:,g);
       w_g  = gw_bulk(g);
    end

    eq = ID(:,A);
    eq = reshape(eq, [], 1);
    K(eq, eq) = K(eq, eq) + Ke;
end

## 4. Matice tvarových funkcí $\mathbf{N}$

**Prompt:**
```
Dej mi funkce v jazyku GNU Octave `bulkShapeFunctions(Xi)` a `bulkShapeFunctionsDerivatives(Xi)`, které pro zadané isoparametrické souřadnice Xi(1) a Xi(2) vrátí řádkové matice funkčních hodnot tvarových funkcí a jejich derivacé pro isoparametrický bilineární čtyřuzlový quad element. Lokální číslování zvol v protisměru hodinových ručiček s počátkem v levém dolním rohu.
```

In [15]:
function N = bulkShapeFunctions(Xi)
    N = (1 + [-1 1 1 -1]*Xi(1)) .* (1 + [-1 -1 1 1]*Xi(2)) / 4;
end
function dNdXi = bulkShapeFunctionsDerivatives(Xi)
    dNdXi = [[-1 1 1 -1]   .* (1 + [-1 -1 1 1]*Xi(2));
        (1 + [-1 1 1 -1]*Xi(1)) .* [-1 -1 1 1]] / 4;
end

In [16]:
K = sparse(neq, neq);

nel = size(IEN, 2); % Number of ELement
for el = 1:nel
    Ke = zeros(nen*nnd, nen*nnd);

    A = IEN(:,el);
    
    for g = 1:ngp
       Xi = gp_bulk(:, g);
       w = gw_bulk(g);
       N = bulkShapeFunctions(Xi_g);
       dNdXi = bulkShapeFunctionsDerivatives(Xi_g);
    end

    eq = ID(:,A);
    eq = reshape(eq, [], 1);
    K(eq, eq) = K(eq, eq) + Ke;
end

## 5. Operátorová matice $\mathbf{B}$

Operátorová matice $\mathbf{B}$ zobrazuje vektor posunutí $\mathbf{u}$ na vektor přetvoření $\left\{ \boldsymbol{\varepsilon}\right\} $:
$$
\left\{ \boldsymbol{\varepsilon}\right\} =\mathbf{B}\mathbf{u}
$$
Pro 2D případ je definována jako:
$$
\left\{ \begin{array}{c}
\varepsilon_{x}\\
\varepsilon_{y}\\
\gamma_{xy}
\end{array}\right\} =\left[\begin{array}{cc}
\frac{\partial}{\partial x} & 0\\
0 & \frac{\partial}{\partial y}\\
\frac{\partial}{\partial y} & \frac{\partial}{\partial x}
\end{array}\right]\left\{ \begin{array}{c}
u\\
v
\end{array}\right\} =\left[\begin{array}{cc}
\frac{\partial}{\partial x} & 0\\
0 & \frac{\partial}{\partial y}\\
\frac{\partial}{\partial y} & \frac{\partial}{\partial x}
\end{array}\right]\left\{ \begin{array}{c}
\sum_{a=1}^{n_{\text{en}}}N_{a}u_{a}\\
\sum_{a=1}^{n_{\text{en}}}N_{a}v_{a}
\end{array}\right\} 
$$
$$
\left\{ \begin{array}{c}
\varepsilon_{x}\\
\varepsilon_{y}\\
\gamma_{xy}
\end{array}\right\} =\left[\left[\begin{array}{cc}
\frac{\partial N_{1}}{\partial x} & 0\\
0 & \frac{\partial N_{1}}{\partial y}\\
\frac{\partial N_{1}}{\partial y} & \frac{\partial N_{1}}{\partial x}
\end{array}\right]\cdots\left[\begin{array}{cc}
\frac{\partial N_{n_{\text{en}}}}{\partial x} & 0\\
0 & \frac{\partial N_{n_{\text{en}}}}{\partial y}\\
\frac{\partial N_{n_{\text{en}}}}{\partial y} & \frac{\partial N_{n_{\text{en}}}}{\partial x}
\end{array}\right]\right]\left\{ \begin{array}{c}
u_{a}\\
v_{a}\\
\vdots\\
u_{n_{\text{en}}}\\
v_{n_{\text{en}}}
\end{array}\right\} 
$$

Následující funkce implementuje tvorbu operátorové matice `B`. Všimněte si, že parametrem této funkce jsou derivace tvarových funkcí podle fyzikálních souřadnic $\boldsymbol{x}$. K dispozici všam máme v tuto chvíli pouze derivace podle isoparametrických souřadnic $\boldsymbol{\xi}$. Potřebujeme proto transformovat suřadnice, viz dále.

In [17]:
function B = createMatrixB(dNdX)
    nsd = size(dNdX, 1); % Number of Spatial Dimensions
    nen = size(dNdX, 2); % Number of Element Nodes
    B = zeros(nsd+1, nsd*nen);
    for a = 1:nen
        B(:, (a-1)*nsd+1:a*nsd ) = [
         dNdX(1, a)     0
            0        dNdX(2, a)
         dNdX(2, a)  dNdX(1, a)];
    end
end

Princip isoparametrických elementů spočívá v tom, že stejná diskretizace, která je použita pro neznámé pole, je použita k popisu fyzikálních souřadnic:
$$
\boldsymbol{u}(\boldsymbol{\xi})=\sum_{a=1}^{n_{\text{en}}}N_{a}(\boldsymbol{\xi})\boldsymbol{u}_{a},\qquad\boldsymbol{x}(\boldsymbol{\xi})=\sum_{a=1}^{n_{\text{en}}}N_{a}(\boldsymbol{\xi})\boldsymbol{x}_{a}
$$
Pro diferenciál fyzikálních sořadnic platí:
$$
\text{d}\boldsymbol{x}=\frac{\partial\boldsymbol{x}}{\partial\boldsymbol{\xi}}\text{d}\boldsymbol{\xi}=\mathbf{J}\text{d}\boldsymbol{\xi}
$$
kde $\mathbf{J}$ se nazývá Jacobiho matice, kterou můžeme pomocí tvarových funkcí a uzlových fyzikálních souřadnic zapsat jako:
$$
\mathbf{J}=\left[\begin{array}{ccc}
x_{1} & \cdots & x_{n_{\text{en}}}\\
y_{1} & \cdots & y_{n_{\text{en}}}
\end{array}\right]\left[\begin{array}{cc}
\frac{\partial N_{1}}{\partial\xi_{1}} & \frac{\partial N_{1}}{\partial\xi_{2}}\\
\vdots & \vdots\\
\frac{\partial N_{n_{\text{en}}}}{\partial\xi_{1}} & \frac{\partial N_{n_{\text{en}}}}{\partial\xi_{2}}
\end{array}\right]
$$

Derivace tvarových funkcí podle fyzikálních souřadnic získáme pravidlem pro derivování složené funkce:
$$
\frac{\partial\mathbf{N}(\boldsymbol{\xi}(\boldsymbol{x}))}{\partial\boldsymbol{x}}=\frac{\partial\boldsymbol{\xi}}{\partial\boldsymbol{x}}\frac{\partial\mathbf{N}}{\partial\boldsymbol{\xi}}=\left[\frac{\partial\boldsymbol{x}}{\partial\boldsymbol{\xi}}\right]^{-1}\frac{\partial\mathbf{N}}{\partial\boldsymbol{\xi}}=\mathbf{J}^{-1}\frac{\partial\mathbf{N}}{\partial\boldsymbol{\xi}}
$$
a zapsáno ve složkách pro všech `nen` složek:
$$
\left[\begin{array}{c}
\frac{\partial N_{1}}{\partial x}\\
\frac{\partial N_{1}}{\partial y}
\end{array}\cdots\begin{array}{c}
\frac{\partial N_{n_{\text{en}}}}{\partial x}\\
\frac{\partial N_{n_{\text{en}}}}{\partial y}
\end{array}\right]=\left[\begin{array}{cc}
\frac{\partial x}{\partial\xi_{1}} & \frac{\partial x}{\partial\xi_{2}}\\
\frac{\partial y}{\partial\xi_{1}} & \frac{\partial y}{\partial\xi_{2}}
\end{array}\right]^{-1}\left[\begin{array}{c}
\frac{\partial N_{1}}{\partial\xi_{1}}\\
\frac{\partial N_{1}}{\partial\xi_{2}}
\end{array}\cdots\begin{array}{c}
\frac{\partial N_{n_{\text{en}}}}{\partial\xi_{1}}\\
\frac{\partial N_{n_{\text{en}}}}{\partial\xi_{2}}
\end{array}\right]
$$

Pro diferenciál elementu můžeme psát:
$$
\text{d}\Omega_{el}=\left\Vert \frac{\partial\boldsymbol{x}}{\partial\xi_{1}}\text{d}\xi_{1}\times\frac{\partial\boldsymbol{x}}{\partial\xi_{2}}\text{d}\xi_{2}\right\Vert =\left\Vert \frac{\partial\boldsymbol{x}}{\partial\xi_{1}}\times\frac{\partial\boldsymbol{x}}{\partial\xi_{2}}\right\Vert \text{d}\xi_{1}\text{d}\xi_{2}=\det(\mathbf{J})\text{d}\xi_{1}\text{d}\xi_{2}
$$

In [18]:
K = sparse(neq, neq);

nel = size(IEN, 2); % Number of ELement
for el = 1:nel
    Ke = zeros(nen*nnd, nen*nnd);

    A = IEN(:, el);
    
    for g = 1:ngp
        Xi = gp_bulk(:, g);
        w  = gw_bulk(g);
        N = bulkShapeFunctions(Xi);
        dNdXi = bulkShapeFunctionsDerivatives(Xi);
       
        Xe = X(:, IEN(:, el) );
        Je = Xe * dNdXi';
        j = det(Je);
        dNdX = inv(Je) * dNdXi;
        Be = createMatrixB(dNdX);
    end

    eq = ID(:,A);
    eq = reshape(eq, [], 1);
    K(eq, eq) = K(eq, eq) + Ke;
end

## 6. Matice materiálové tuhosti $\mathbf{D}$

Matice materiálové tuhosti je lineární operátor, který vektor přetvoření $\left\{ \boldsymbol{\varepsilon}\right\}$ zobrazí na vektor napětí $\left\{ \boldsymbol{\sigma}\right\}$: 
$$
\left\{ \boldsymbol{\sigma}\right\} =\mathbf{D}\left\{ \boldsymbol{\varepsilon}\right\} 
$$
podobně jako to dělá konstanta úměrnosti (Youngův modul pružnosti) $E$ v 1D podobě Hookeova zákona: 
$$
\sigma=E\varepsilon
$$
Ve 2D je potřeba rozlišovat mezi případem rovinné **deformace** a rovinné **napjatosti** viz poznámky nebo následující prompt pro LLM.

**Prompt:**
```
Dej mi matici materiálové tuhosti D pro případ rovinné deformace zapsanou v syntaxi GNU Octave.
```

In [19]:
E = 210e9;   % Youngův modul [Pa]
nu = 0.3;    % Poissonovo číslo [-]
D = E / ((1 + nu) * (1 - 2*nu)) * [1-nu,  nu,       0;
                                   nu,    1-nu,     0;
                                   0,     0,    (1-2*nu)/2];

In [20]:
K = sparse(neq, neq);

nel = size(IEN, 2); % Number of ELement
for el = 1:nel
    Ke = zeros(nen*nnd, nen*nnd);

    A = IEN(:, el);
    
    for g = 1:ngp
        Xi = gp_bulk(:, g);
        w  = gw_bulk(g);
        N = bulkShapeFunctions(Xi);
        dNdXi = bulkShapeFunctionsDerivatives(Xi);
       
        Xe = X(:, IEN(:, el) );
        Je = Xe * dNdXi';
        j = det(Je);
        dNdX = inv(Je) * dNdXi;
        Be = createMatrixB(dNdX);
        
        Ke = Ke + Be'*D*Be * w * j
    end

    eq = ID(:,A);
    eq = reshape(eq, [], 1);
    K(eq, eq) = K(eq, eq) + Ke;
end




























Ke =
 Columns 1 through 6:
  -5.8252e+10   3.1399e+10   4.3144e+10  -7.5117e+09   1.5609e+10  -8.4135e+09
   3.1399e+10  -5.4943e+10  -1.5474e+10   2.0648e+09  -8.4135e+09   1.4722e+10
   4.3144e+10  -1.5474e+10  -4.7193e+10  -8.4135e+09  -1.1561e+10   4.1463e+09
  -7.5117e+09   2.0648e+09  -8.4135e+09  -1.6233e+10   2.0128e+09  -5.5326e+08
   1.5609e+10  -8.4135e+09  -1.1561e+10   2.0128e+09  -4.1823e+09   2.2544e+09
  -8.4135e+09   1.4722e+10   4.1463e+09  -5.5326e+08   2.2544e+09  -3.9447e+09
  -5.0073e+08  -7.5117e+09   1.5609e+10   1.3912e+10   1.3417e+08   2.0128e+09
  -1.5474e+10   3.8156e+10   1.9741e+10   1.4722e+10   4.1463e+09  -1.0224e+10
 Columns 7 and 8:
  -5.0073e+08  -1.5474e+10
  -7.5117e+09   3.8156e+10
   1.5609e+10   1.9741e+10
   1.3912e+10   1.4722e+10
   1.3417e+08   4.1463e+09
   2.0128e+09  -1.0224e+10
  -1.5242e+10  -8.4135e+09
  -8.4135e+09  -4.2654e+10

Ke =
 Columns 1 through 6:
  -1.0544e+11   3.9813e+10   8.6289e+10   7.9626e+09

## 7. Okrajové podmínky
### Dirichletovy okrajové podmínky
2D pole `DBC`


In [21]:
DBC = [1 1 0.0
       1 2 0.0
       2 1 0.0
       2 2 0.0];

### Neumannovy okrajové podmínky
Zadání tohoto typu okrajové podmínky implementujeme pomocí 2D pole `NBC`. Každý řádek tohoto pole obsahuje čtyři údaje:
1. `elementID`: číslo elementu
2. `edgeID`: lokální číslo hrany, na kterou se trakce aplikuje (1, 2, 3, nebo 4)
3. `t_x`: x-ová složka trakčního vektoru v globálním souřadicovém systému
4. `t_y`: y-ová složka trakčního vektoru v globálním souřadicovém systému

<div class="homework">
<strong>Úkol:</strong> Zkus změnit okrajovou podmínku tak, že prut nebude namáhán na tah, ale na smyk.
</div>

In [22]:
NBC = [3 1 0 -0.1]; % elementID = 3, edgeID = 1, tx = 0, t_y = -0.1 Pa

In [24]:
f = zeros(neq, 1);

ISN = [2 3 4 1
       1 2 3 4];

for i = 1:size(NBC, 1)
    el = NBC(i, 1);
    sg = NBC(i, 2);
    tx = NBC(i, 3);
    ty = NBC(i, 4);
    
    fe = zeros(nen*nnd, 1);

    A = IEN(ISN(:, sg), el);
    
    for g = 1:ngp
        Xi = gp_boundary(:, g);
        w  = gw_boundary(g);
        N = boundaryShapeFunctions(Xi);
        dNdXi = boundaryShapeFunctionsDerivatives(Xi);
       
        Xe = X(:, IEN(ISN(:, sg), el) );
        Je = Xe * dNdXi';
        j = norm(cross(Je(1,:), Je(2,:)));
        dNdX = inv(Je) * dNdXi;

        fe = fe + kron(N',[tx; ty]) * w * j
    end

    eq = ID(:,A);
    eq = reshape(eq, [], 1);
    f(eq) = f(eq) + fe;
end


































error: 'gp_boundary' undefined near line 12, column 14


## 8. Řešení

## 9. Post-processing